In [9]:
import numpy
import cv2
import math

In [10]:
interpolationMethods = {
         0: cv2.INTER_NEAREST, 
         1: cv2.INTER_LINEAR, 
         2: cv2.INTER_CUBIC
}

In [11]:
def open_img(img):
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
def rotate_img(img, angle):
 
    angle %=360
 
    # http://qaru.site/questions/274040/image-size-python-opencv
    (height, width) = img.shape[:2]
    (center_x, center_y) = (width // 2, height // 2)
    cos = numpy.cos(angle)
    sin = numpy.sin(angle)
 
    M = cv2.getRotationMatrix2D((center_x, center_y), angle, 1.0)
    
    #http://qaru.site/questions/121230/rotate-image-and-crop-out-black-borders
    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    new_width = width * math.cos(alpha) + height * math.sin(alpha)
    new_height = width * math.sin(alpha) + height * math.cos(alpha)

    gamma = math.atan2(new_width, new_width) if (width < height) else math.atan2(new_height, new_height)

    delta = math.pi - alpha - gamma

    length = height if (width < height) else width

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    new_width = int(new_width - 2 * x)
    new_height = int(new_height - 2 * y)

    M[0, 2] += (new_width/ 2) - center_x
    M[1, 2] += (new_height / 2) - center_y
    
    img = cv2.warpAffine(img, M, (new_width, new_height))
    return img

In [ ]:
img = cv2.imread(input("Введите абсолютный путь до изображения: ")) 
angle = int(input("Введите угол поворота (в градусах): "))
k = float(input("Введите коэффициент сжатия из диапазона (0;1]: "))
interpolation = int(input("Выберите признак интерполяционной схемы (0 – нулевого порядка, 1 – первого порядка, 2 - третьего порядка): "))
new_img= rotate_img(img, angle)
result_img = cv2.resize(new_img, None, fx =k, fy=1/k,interpolation=interpolationMethods[interpolation])
result_path = input("Ввведите путь для сохранения изображения: ")
cv2.imwrite(result_path + "/result.bmp", result_img);
#cv2.imwrite("result.bmp", result_img);
open_img(result_img)

Введите абсолютный путь до изображения: /Users/xtail/Projects/LETI/ImageProcessing/labs/lab1/test_files/2.jpg
Введите угол поворота (в градусах): 75
Введите коэффициент сжатия из диапазона (0;1]: 0.45
Выберите признак интерполяционной схемы (0 – нулевого порядка, 1 – первого порядка, 2 - третьего порядка): 0
Ввведите путь для сохранения изображения: /Users/xtail/Projects/LETI/ImageProcessing/labs/lab1/result
